## Paparella Results
Paparella et. al. published results of their models only for  
certain time-points and each model was evaluated in different  
time point selected based on ndcg@10 value. However, vanilla models  
hit their highest accuracy early on in training (while diversity and  
novelty are still growing), while SMORL models hit their highest  
accuracy much later in training process (where diversity and novelty  
reach their maximums). This cause that diversity, novelty and likely  
also repetitiveness of vanilla models may be underestimated.

#### Published and "Published" data 
As mentioned above, Paprella published metrics only for certain time points of training  
process in the paper (i.e. in tables and graphs). However, authors also published code  
with various data files including text files that contains almost time series of metrics  
from training of model. I wrote almost as it is plain text that needs to be further processed  
to get time series. 

This notebook iterates through .txt output files, processes them and provide time series of  
accuracy, diversity, novelty and repetitiveness metrices as well as loss.

In [ ]:
import re
from torch.utils.tensorboard import SummaryWriter

In [ ]:
patterns = {
    # Matches: cumulative reward @ 5: 8585.000000
    'cumulative_reward': re.compile(r'^cumulative reward @ (\d+): ([\d.]+)$'),

    # Matches: clicks hr ndcg @ 10 : 0.398485, 0.241570
    'clicks_hr_ndcg': re.compile(r'^clicks hr ndcg @ (\d+) ?: ([\d.]+), ([\d.]+)$'),

    # Matches: purchase hr and ndcg @10 : 0.535500, 0.337177
    'purchase_hr_ndcg': re.compile(r'^purchase hr and ndcg @(\d+) ?: ([\d.]+), ([\d.]+)$'),

    # Matches: total diversity reward: 48740.039062
    'total_diversity': re.compile(r'^total diversity reward: ([\d.]+)$'),

    # Matches: total novelty reward: 19004.000000
    'total_novelty': re.compile(r'^total novelty reward: ([\d.]+)$'),

    # Matches: coverage of top 5 predictions: 0.379410
    'coverage': re.compile(r'^coverage of top (\d+) predictions: ([\d.]+)$'),

    # Matches: coverage on novel items of top 10 predictions: 0.385095
    'novel_coverage': re.compile(r'^coverage on novel items of top (\d+) predictions: ([\d.]+)$'),

    # Matches: average number of repetitions in top 20: 53.379500
    'avg_repetitions': re.compile(r'^average number of repetitions in top (\d+): ([\d.]+)$'),
}

In [ ]:
models = ['sasrec', 'caser', 'gru']
combs = [[1,1,1],[1,1,0],[1,0,1],[0,1,1],[0,1,0],[0,0,1]]
model_stop = ["main", "target"]

def process_results(data_path, results_path, variant):
    writer = SummaryWriter(log_dir=results_path)
    with open(data_path, "r") as f:
        print(f"OPENED {data_path}")
        lines = f.readlines()
        my_model = "any"  
        for line in lines:
            match = re.search(r"Step: (\d+)\.+\s+Loss: ([\d.]+)", line)
            if match:
                step = int(match.group(1))
                loss = float(match.group(2))
                writer.add_scalar(f"Loss/train", loss, step)
                print(f"Loss/train: {loss}, {step}")
            if "Model is" in line:
                my_model = "any"
            if "Evaluating Target Model" in line:
                my_model = "target"
            if "Evaluating Main Model" in line:
                my_model = "main"
            if my_model == variant:
                continue   
            if "TEST" in line:
                continue
            match = re.search(r"Step: (\d+)\.+\s+Loss: ([\d.]+)", line)
            if match:
                step = int(match.group(1))
                loss = float(match.group(2))
                writer.add_scalar(f"Loss/train", loss, step)
            if m := patterns['clicks_hr_ndcg'].match(line):
                k, hr, ndcg = int(m.group(1)), float(m.group(2)), float(m.group(3))
                writer.add_scalar(f"HR_0/{k}", hr,  step)
                writer.add_scalar(f"NDCG/{k}", ndcg,  step)
            #elif m := patterns['total_diversity'].match(line):
            #    print("Total diversity reward:", float(m.group(1)))
            #elif m := patterns['total_novelty'].match(line):
            #    print("Total novelty reward:", float(m.group(1)))
            elif m := patterns['coverage'].match(line):
                k, val = int(m.group(1)), float(m.group(2))
                writer.add_scalar(f"COV/{k}", val,  step)
            #    print("Coverage of top {}: {}".format(k, val))
            elif m := patterns['novel_coverage'].match(line):
                k, val = int(m.group(1)), float(m.group(2))
                writer.add_scalar(f"NOV/{k}", val,  step)
            #    print("Novel coverage of top {}: {}".format(k, val))
            elif m := patterns['avg_repetitions'].match(line):
                k, val = int(m.group(1)), float(m.group(2))
                writer.add_scalar(f"REP/{k}", val,  step)
            #    print("Avg repetitions in top {}: {}".format(k, val))          
            #elif m := patterns['cumulative_reward'].match(line):
            #    k, val = int(m.group(1)), float(m.group(2))
            #    print("Reward@{} = {}".format(k, val))
            #elif m := patterns['purchase_hr_ndcg'].match(line):
            #    k, hr, ndcg = int(m.group(1)), float(m.group(2)), float(m.group(3))
            #    print("Purchase HR@{} = {}, NDCG@{} = {}".format(k, hr, k, ndcg))
            #elif m := patterns['total_diversity'].match(line):
            #    print("Total diversity reward:", float(m.group(1)))
            #elif m := patterns['total_novelty'].match(line):
            #    print("Total novelty reward:", float(m.group(1)))
    writer.flush()
    writer.close()


for model in models:
    for comb in combs:
        for variant in model_stop:
            file_path = f'/home/marek/Kinit/MORSs/SMORL/div4rec/rc15_results/{model}_smorl/{model}_smorl1_acc{comb[0]}.0_div{comb[1]}.0_nov{comb[2]}.0_weighted_q_vals.txt'
            results = f"runs/Paparella_2/{model}/rl_{comb[0]}{comb[1]}{comb[2]}_{variant}"
            print(f"Processing: {model}-{comb}-{variant}")
            process_results(file_path, results, variant)


for model in models:
    file_path = f'/home/marek/Kinit/MORSs/SMORL/div4rec/rc15_results/{model}/{model}.txt'
    results = f"runs/Paparella_2/{model}/base"
    process_results(file_path, results, "This is not rl - not distinguished")